# Outline

1.) Color Change Detection in video, use same frame as rest of video. (Needs to be done on seperate instance of picture/frame as detections as bounding box may cause issues with color.
2.) Use premade model for the moment to genereate bounding boxes in frames
3.) Set lines in model to 
3.) Use bounding box boundaries to determine crossings against 

In [1]:
import cv2
import time
import numpy as np
import pandas as pd
from ultralytics import YOLO
# import supervision as sv

In [2]:
car_x, car_y, car_x2, car_y2, Matchpersent, indexModule = -1, -1, -1, -1, -1, -1
def display_roi(row_data):
    global car_x, car_y, car_x2, car_y2, Matchpersent, indexModule
    car_x, car_y, car_x2, car_y2, Matchpersent, indexModule = int(row_data[0]), int(row_data[1]), int(row_data[2]), int(row_data[3]), int(row_data[4] * 100), int(row_data[5])
    

In [3]:
roi_x, roi_y, roi_width, roi_height = -1, -1, -1, -1
selecting_roi = False

def select_roi(event, x, y, flags, param):
    global roi_x, roi_y, roi_width, roi_height, selecting_roi

    if event == cv2.EVENT_LBUTTONDOWN:
        roi_x, roi_y = x, y
        selecting_roi = True

    elif event == cv2.EVENT_LBUTTONUP:
        roi_width, roi_height = x - roi_x, y - roi_y
        selecting_roi = False


In [4]:
roi_x1, roi_y1, roi_width1, roi_height1 = -1, -1, -1, -1

def select_roi1(event, x, y, flags, param):
    global roi_x1, roi_y1, roi_width1, roi_height1, selecting_roi

    if event == cv2.EVENT_LBUTTONDOWN:
        roi_x1, roi_y1 = x, y
        selecting_roi = True

    elif event == cv2.EVENT_LBUTTONUP:
        roi_width1, roi_height1 = x - roi_x1, y - roi_y1
        selecting_roi = False

In [5]:
roi_x2, roi_y2, roi_width2, roi_height2 = -1, -1, -1, -1

def select_roi2(event, x, y, flags, param):
    global roi_x2, roi_y2, roi_width2, roi_height2, selecting_roi

    if event == cv2.EVENT_LBUTTONDOWN:
        roi_x2, roi_y2 = x, y
        selecting_roi = True

    elif event == cv2.EVENT_LBUTTONUP:
        roi_width2, roi_height2 = x - roi_x2, y - roi_y2
        selecting_roi = False

In [6]:
roi_x3, roi_y3, roi_width3, roi_height3 = -1, -1, -1, -1

def select_roi3(event, x, y, flags, param):
    global roi_x3, roi_y3, roi_width3, roi_height3, selecting_roi

    if event == cv2.EVENT_LBUTTONDOWN:
        roi_x3, roi_y3 = x, y
        selecting_roi = True

    elif event == cv2.EVENT_LBUTTONUP:
        roi_width3, roi_height3 = x - roi_x3, y - roi_y3
        selecting_roi = False

In [7]:
line_x, line_y, line_x1, line_y1 = -1, -1, -1, -1
drawing = False

def select_line(event, x, y, flags, param):
    global line_x, line_y, line_x1, line_y1, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        line_x, line_y = x, y
        drawing = True

    elif event == cv2.EVENT_LBUTTONUP:
        line_x1, line_y1 = x, y
        drawing = False

In [8]:
def color_grab(frame, roi_y, roi_x, roi_width, roi_height):
    roi = frame[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]

    # Convert the ROI to the desired color space (e.g., RGB or HSV)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

    # Compute the mean color of the ROI
    mean_color = np.mean(roi, axis=(0, 1))
        
    return mean_color

In [9]:
def check_overlap(box_coordinate,line_start, line_end):
    #calcukate the slope
    global testing
    testing = box_coordinate
    m = (line_end[1] - line_start[1])/(line_end[0]-line_start[0])
    b = (line_start[1] - m*line_start[0])
    x2 = box_coordinate[:,0]
    y2 = box_coordinate[:,1]  
    x1 = box_coordinate[:,2]
    y1 = box_coordinate[:,3]    
    
    overlap  = False 
    results = []
    i = 0
    while i < len(box_coordinate):
        if (line_start[0] < x1[i] or line_start[0] < x2[i]):
            if (line_end[0] > x1[i] or line_end[0] > x2[i]):
                if m*x1[i]+b < y1[i] and m*x2[i]+b > y1[i]:
                    overlap = True
                elif m*x1[i]+b > y1[i]and m*x2[i]+b < y1[i]:
                    overlap = True
                else:
                    overlap = False
            else:
                overlap = False
        else:
            overlap = False

        i+=1
        results.append(overlap)
    results = np.array(results)

    return results

In [10]:
#Delay in miliseconds/Setup video
frame_delay = 145000
frame_width = 1100
frame_height = 700

#RGB color to be compared to to check for a shift, assuming signals default is black, in which case 0,0,0 was used
reference_color = (0, 0, 0)  # Black

threshold = 100

#Pixels for window to open from top left of window
opening_x = 10
opening_y = 10
video_dir = r"C:\Users\brema\Desktop\GX020001.MP4"

   
size = (frame_width, frame_height)

#Opens video and starts from desired time after start
video_capture = cv2.VideoCapture(video_dir)
video_capture.set(cv2.CAP_PROP_POS_MSEC, frame_delay)
resultVid = cv2.VideoWriter(r'C:\Users\magic\Desktop\test.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

In [11]:
#Opens frame at chosen start to draw the ROI of the top left signal
ret, frame = video_capture.read()
cv2.namedWindow('Select ROI - Top Left Signal')
cv2.setMouseCallback('Select ROI - Top Left Signal', select_roi)
frame = cv2.resize(frame, (frame_width, frame_height))

while True:
    cv2.imshow('Select ROI - Top Left Signal', frame)
    cv2.moveWindow('Select ROI - Top Left Signal', opening_x, opening_y)
    
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q') or not ret:
        break

    if not selecting_roi and roi_width > 0 and roi_height > 0:
        cv2.destroyWindow('Select ROI - Top Left Signal')
        break

In [12]:
#Opens frame at chosen start to draw the ROI of the top right signal
ret, frame = video_capture.read()
cv2.namedWindow('Select ROI - Top Right Signal')
cv2.setMouseCallback('Select ROI - Top Right Signal', select_roi1)
frame = cv2.resize(frame, (frame_width, frame_height))

while True:
    cv2.imshow('Select ROI - Top Right Signal', frame)
    cv2.moveWindow('Select ROI - Top Right Signal', opening_x, opening_y)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q') or not ret:
        break

    if not selecting_roi and roi_width1 > 0 and roi_height1 > 0:
        cv2.destroyWindow('Select ROI - Top Right Signal')
        break

In [13]:
#Opens frame at chosen start to draw the ROI of the bottom signal
ret, frame = video_capture.read()
cv2.namedWindow('Select ROI - Bottom Signal')
cv2.setMouseCallback('Select ROI - Bottom Signal', select_roi2)
frame = cv2.resize(frame, (frame_width, frame_height))

while True:
    cv2.imshow('Select ROI - Bottom Signal', frame)
    cv2.moveWindow('Select ROI - Bottom Signal', opening_x, opening_y)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q') or not ret:
        break

    if not selecting_roi and roi_width2 > 0 and roi_height2 > 0:
        cv2.destroyWindow('Select ROI - Bottom Signal')
        break

In [14]:
#Opens frame at chosen start to draw the ROI for YOLO detections
ret, frame = video_capture.read()
cv2.namedWindow('Select ROI - YOLO Detections')
cv2.setMouseCallback('Select ROI - YOLO Detections', select_roi3)
frame = cv2.resize(frame, (frame_width, frame_height))

while True:
    cv2.imshow('Select ROI - YOLO Detections', frame)
    cv2.moveWindow('Select ROI - YOLO Detections', opening_x, opening_y)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q') or not ret:
        break

    if not selecting_roi and roi_width3 > 0 and roi_height3 > 0:
        cv2.destroyWindow('Select ROI - YOLO Detections')
        break

In [15]:
#Opens frame at chosen start to draw the Line for crossings
start_pt1, end_pt1, start_pt2, end_pt2 = -1, -1, -1, -1
for i in range (2):
    global start_pt1, end_pt1, start_pt2, end_pt2
    while True:
        ret, frame = video_capture.read()
        cv2.namedWindow('Select Line')
        cv2.setMouseCallback('Select Line', select_line)
        frame = cv2.resize(frame, (frame_width, frame_height))
        cv2.imshow('Select Line', frame)
        cv2.moveWindow('Select Line', opening_x, opening_y)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('q') or not ret:
            break

        if not drawing and line_x1 > 0 and line_y1 > 0:
            cv2.destroyWindow('Select Line')
            break
    if (i == 0):
        start_pt1 = (line_x, line_y)
        end_pt1 = (line_x1, line_y1)
    if (i == 1 ):
        start_pt2 = (line_x, line_y)
        end_pt2 = (line_x1, line_y1)
        
    line_x, line_y, line_x1, line_y1 = -1, -1, -1, -1

In [16]:
#YOLO detection section/declare model
model = YOLO("yolov8n.pt")

#Codes for all detections, assuming relev_codes are veh+peds/bikes and ped_codes are peds and bikes, etc.
relev_codes = [0, 1, 2, 3, 5, 7]
ped_codes = [0]
bike_codes = [1]
veh_codes = [2, 3, 5, 7]

# box_annotator = sv.BoxAnnotator(
#     thickness=1,
#     text_thickness=1,
#     text_scale=0.5
# )


In [17]:
# #How to get class ids
# model.model.names
# #Relevant ids for current model:
# # 0: 'person',
# # 1: 'bicycle',
# # 2: 'car',
# # 3: 'motorcycle',
# # 5: 'bus',
# # 7: 'truck',

In [18]:
# work on saveing the video
# Process the video using the selected ROIs

frame_count = 0
skip_frames = 1 #Added due to slow processing speed, can be removed if processing sped up, 1 representing every other frame, 2 every third frame, etc.
skip_count = 0 #Start of skipped frames, must be less than or equal to skip_frames
display = True #Shows annotations on video output
start_time = time.time() 
pre_data = np.empty((0, 14)) #np.array for data as processed

video_capture = cv2.VideoCapture(video_dir)
video_capture.set(cv2.CAP_PROP_POS_MSEC, frame_delay)
while video_capture.isOpened():
    ret, frame = video_capture.read()

    if not ret:
        break

    if skip_count == skip_frames:
        frame = cv2.resize(frame, (frame_width, frame_height))
    
        
        #YOLOv8 section
        roi3 = frame[roi_y3:roi_y3+roi_height3, roi_x3:roi_x3+roi_width3]
        # roi3 = cv2.cvtColor(roi3, cv2.COLOR_BGR2RGB)
        result = model.predict(roi3, classes = relev_codes, agnostic_nms=False, verbose = False, device = 'cpu')[0]
        #detections = result.box.xyxy[0]
        detectionsCoardinate = np.array(result.boxes.data)
        detectionsCoardinate += (roi_x3,roi_y3,roi_x3,roi_y3,0,0)
        
        
    
        #Get YOLO box coordinates
#         detections_All = np.column_stack((detectionsCoardinate, detections.class_id))
        detections_Peds = np.array(detectionsCoardinate[np.isin(detectionsCoardinate[:, 5], ped_codes)])
        detections_Bikes = np.array(detectionsCoardinate[np.isin(detectionsCoardinate[:, 5], bike_codes)])
        detections_Veh = np.array(detectionsCoardinate[np.isin(detectionsCoardinate[:, 5], veh_codes)])

    
        # Check overlap
        overlap_veh1 = np.any(check_overlap(detections_Veh, start_pt1, end_pt1))
        overlap_veh2 = np.any(check_overlap(detections_Veh, start_pt2, end_pt2))
        overlap_ped1 = np.any(check_overlap(detections_Peds, start_pt1, end_pt1))
        overlap_ped2 = np.any(check_overlap(detections_Peds, start_pt2, end_pt2))

        overlap_veh = False 
        overlap_ped = False
        overlap_bike = False
        if (overlap_veh1 == 1 )or (overlap_veh2 == 1):
            overlap_veh = True
        # if (overlap_bike1 == 1 ) or (overlap_bike2 == 1):
        #     overlap_bike == True
        if (overlap_ped1 == 1) or (overlap_ped2 == 1):
            overlap_ped = True
            
        
        #Gets timestamp of frame in video in miliseconds
        timestamp = video_capture.get(cv2.CAP_PROP_POS_MSEC)
    
        print(testing)
        
        PedCross = ''
        VehCross = 'Vehicle not-crossing'
#         BikeCross = ''
    
        if overlap_ped:
            PedCross = 'Person Crossing'
        if overlap_veh:
            VehCross = 'Vehicle Crossing'
        # if overlap_bike == 1 :
        #     BikeCross = 'Bike Crossing'
    
        frame_count += 1
    
        skip_count = 0
        
    
    
    
#         if display:
#         #Displays YOLO boxes
# #             frame = box_annotator.annotate(
# #                 scene=frame,
# #                 detectionsCoardinate=detectionsCoardinate,
# #                 labels=labels
# #             )
            
#             # Draw rectangles around the ROIs on the frame
#             cv2.rectangle(frame, (roi_x, roi_y), (roi_x+roi_width, roi_y+roi_height), (0, 255, 0), 2)
#             cv2.rectangle(frame, (roi_x1, roi_y1), (roi_x1+roi_width1, roi_y1+roi_height1), (0, 255, 0), 2)
#             cv2.rectangle(frame, (roi_x2, roi_y2), (roi_x2+roi_width2, roi_y2+roi_height2), (0, 255, 0), 2)
#             cv2.line(frame, start_pt, end_pt, (128, 0, 128), 2)
#             elapsed_time = time.time() - start_time
#             fps = frame_count/elapsed_time
#             cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
#             cv2.putText(frame, f"TIME: {timestamp:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
#             cv2.putText(frame, f"TL:"+ TLShift, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
#             cv2.putText(frame, f"TR:"+ TRShift, (10, 120), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
#             cv2.putText(frame, f"BOT:"+ BotShift, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
#             cv2.putText(frame, f"Ped Cross:"+ PedCross, (10, 180), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
#             cv2.putText(frame, f"Veh Cross:"+ VehCross, (10, 210), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
#             cv2.putText(frame, f"Bike Cross:"+ BikeCross, (10, 240), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)

        for i in detectionsCoardinate:
            display_roi(i)
            
            if indexModule == 2:
                cv2.putText(frame, 'Car ' + str(Matchpersent), (car_x, car_y - 3), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 115), 1)
            if indexModule == 0:
                cv2.putText(frame, 'Person ' + str(Matchpersent), (car_x, car_y - 3), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 115), 1)
                
            cv2.rectangle(frame, (car_x, car_y), (car_x2, car_y2), (255, 255, 255), 2)
        cv2.rectangle(frame, (roi_x, roi_y), (roi_x+roi_width, roi_y+roi_height), (0, 255, 0), 2)
        cv2.rectangle(frame, (roi_x1, roi_y1), (roi_x1+roi_width1, roi_y1+roi_height1), (0, 255, 0), 2)
        cv2.rectangle(frame, (roi_x2, roi_y2), (roi_x2+roi_width2, roi_y2+roi_height2), (0, 255, 0), 2)
        cv2.line(frame, start_pt1, end_pt1, (128, 0, 128), 2)
        cv2.line(frame, start_pt2, end_pt2, (128, 0, 128), 2)
        elapsed_time = time.time() - start_time
        fps = frame_count/elapsed_time
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
        cv2.putText(frame, f"TIME: {timestamp:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        cv2.putText(frame, f"Ped Cross:"+ PedCross, (10, 180), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
        cv2.putText(frame, f"Veh Cross:"+ VehCross, (10, 210), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255), 2)
        
        
        # Display the frame
        cv2.imshow('Video', frame)        
        resultVid.write(frame)

    else:
        frame_count += 1
        skip_count += 1

    #Breaks the video early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
video_capture.release()
resultVid.release()
cv2.destroyAllWindows()
print('alldone')

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[        870      268.69      905.77      309.84     0.28253           0]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
alldone


In [19]:
m = (end_pt1[1] - start_pt1[1])/(end_pt1[0]-start_pt1[0])
b = (start_pt1[1] - m*start_pt1[0])
x1 = detections_Veh[:,0]
y1 = detections_Veh[:,1]  
x2 = detections_Veh[:,2]
y2 = detections_Veh[:,3]    
overlap  = False 
results = []
i = 0
while i < len(detections_Veh):
    if m*x2[i]+b < y2[i] and m*x1[i]+b > y2[i]:
        overlap = True
    elif m*x2[i]+b > y2[i]and m*x1[i]+b < y1[i]:
        overlap = True
    else:
        overlap = False
    i+=1
    results.append(overlap)
    print(overlap)
results = np.array(results)
print (results)


[]


In [20]:
test = np.array(detectionsCoardinate[np.isin(detectionsCoardinate[:, 5], veh_codes)])
print(np.any(check_overlap(detections_Veh, start_pt1, end_pt1)))
print(np.any(check_overlap(detections_Veh, start_pt2, end_pt2)))

False
False


In [21]:
detectionsCoardinate

array([], shape=(0, 6), dtype=float32)

In [22]:
#Building pandas dataframe from date extracted from video feed.
STL1 = np.where(np.any(np.abs(np.subtract(reference_color, pre_data[:, [2,3,4]])) > threshold, axis = 1),1,0)
STR1 = np.where(np.any(np.abs(np.subtract(reference_color, pre_data[:, [5,6,7]])) > threshold, axis = 1),1,0)
SB1 = np.where(np.any(np.abs(np.subtract(reference_color, pre_data[:, [8,9,10]])) > threshold, axis = 1),1,0)

df_Data = pd.DataFrame({
    'Frame': pre_data[:,0], 
    'Time': pre_data[:,1], 
    'PHB Phase Code': np.zeros(len(pre_data)), 
    'Users Event Code': np.zeros(len(pre_data)),  
    'Car Cross': pre_data[:,11], 
    'Ped Cross': pre_data[:,12],
    'Bike Cross': pre_data[:,13]
})


pre_Check = True
yellow = False
solid = False
flash_Red = False
df_Data['Cur Phase'] = 0
df_Data['Phase'] = 0

#Overlap of all lights

i = 0

#Gets proper phases, assuming data starts from an off phase
while i < len(df_Data):
    
    if df_Data.loc[i, 'Cur Phase'] == 1 and pre_Check:
        j = i
        pre_Check = False
        yellow = True
        df_Data.loc[i, 'Phase'] = 1
    elif df_Data.loc[i, 'Cur Phase'] == 1:
        if change_Phase:
            df_Data.loc[i, 'Phase'] = 2
        else:
            df_Data.loc[i, 'Phase'] = 1
    elif df_Data.loc[i, 'Cur Phase'] == 0:
        pre_Check = True
        change_Phase = False
        flash_Red = False
        if yellow:
            df_Data.loc[i, 'Phase'] = 1
    elif df_Data.loc[i, 'Cur Phase'] == 2 | df_Data.loc[i, 'Cur Phase'] == 3 and not change_Phase and not pre_Check:
        i = j - 1
        change_Phase = True
        yellow = False
        solid = True
    elif solid:
        if df_Data.loc[i, 'Cur Phase'] == 2:
            df_Data.loc[i, 'Phase'] = 3          
            solid = False
        elif df_Data.loc[i, 'Cur Phase'] == 3:
            df_Data.loc[i, 'Phase'] = 3
    elif df_Data.loc[i, 'Cur Phase'] == 2 and change_Phase:
        if flash_Red:
            df_Data.loc[i, 'Phase'] = 4
        else:
            df_Data.loc[i, 'Phase'] = 3
    elif df_Data.loc[i, 'Cur Phase'] == 3:
        df_Data.loc[i, 'Phase'] = 4
        flash_Red = True
        
    i += 1

#Bin up data

    

In [23]:
#Check a frame
check_frame = False

#Time of desired frame to be checked
frame_delay = 70303.5666666667

#Pulls up frame desired to be checked
if check_frame:
    video_capture = cv2.VideoCapture(video_dir)
    video_capture.set(cv2.CAP_PROP_POS_MSEC, frame_delay)

    
    # Read the frame at the desired position
    ret, frame = video_capture.read()
    
    # Check if the frame was read successfully
    if ret:
        # Display the frame
        frame = cv2.resize(frame, (frame_width, frame_height))
        cv2.imshow('Frame', frame)
        cv2.waitKey(0)
    else:
        print("Unable to read the frame.")
    
    video_capture.release()
    cv2.destroyAllWindows()

In [24]:
#Check Video
#When run, true or false dictates if a frame to be checked should be pulled up
check_frame = False

#Time of desired frame to be checked
frame_delay = 84451.0333333333

if check_frame:
    video_capture = cv2.VideoCapture(video_dir)
    video_capture.set(cv2.CAP_PROP_POS_MSEC, frame_delay)
    
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        
        
        if not ret:
            break
        frame = cv2.resize(frame, (frame_width, frame_height))
        # Display the frame with the ROI
        cv2.imshow('Video', frame)
    
        #Breaks the video early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the video capture and close all windows
    video_capture.release()
    cv2.destroyAllWindows()